# SA CCR - Allocation and Aggregation of intruments

**Introduction:<br>**
This notebook contains the methods for allocating instruments into the correct categories of risk for the SA CCR methodology for the determination of counterparty credit risk capital requirements. SA CCR was introduced in 2014 and phased in by January 2017 as part of the Basel III framework. It replaced Basel I Current Exposure Method (CEM) and Basels II Standardized Method (SM).<br>


<u>Source</u> : https://www.bis.org/basel_framework/chapter/CRE/52.htm?inforce=20230101&published=20200605#paragraph_CRE_52_20230101_52_24


**Model Summary:<br>**

At a high level interpretation, the SA CCR methodology can be broken down into the following steps :
- A top down allocation of instruments to corresponding {Asset Class x Hedging Set x Bucket} category:
    - Netting set -> Asset Class -> Hedging Set -> Bucket -> instrument
- Mapping of instruments to the parametors required to calculate the Effective Notional of the instrument (more or less delta sensitity to underlying RF):
    -  instrument -> instrument parametors -> Effective Notional of transaction
- A bottom up aggregation of instrument Effective Notionals to obtain the netting set EAD:
    - Effective Notional of instrument -> Bucket -> Hedging Set -> Asset Class -> Netting set
    
<br>
This Notebook concerns itself with the first and trhid stepss: 
- allocation of instruments to the correct categories of risk
- botton up aggregation of results 


**Author:** <br>
This Notebook (implementation and analysis) is the work and property of Joshua KAJI - FRM. All Rights reserved.

**Structure of the document:**<br>
The notebook is structured to facilitate comprehension of the methodology, both in terms of implementation and description. Each code cell is preceded by an explanatory note outlining the tasks undertaken, as well as the subsection reference (eg: CRE52.XX) to which it relates in the regulatory documentation.

**Table of Content:**
1) [Brief overview of the SA CCR methodology and instrument parametors required for compuation EAD for each netting set](#section1)<br>
    1.1 [EAD as a function of current exposure (RC) and potential future exposure (PFE)](#section2)<br>
    1.2 [General steps followed for computing AddOn(asset class)](#section3)<br>
2) [Intrument superclass and subclasses](#section4)<br>
3) [Hedging Set superclass and hedging set specific subclasses](#section4)<br>
    3.1 [IR Hedging Set class](#section5)<br>
    3.2 [FX Hedging Set class](#section6)<br>
    3.3 [Equity Hedging Set class](#section7)<br>
4) [Asset class superclass and asset class specific subclasses](#section8)<br>
    4.1 [IR class](#section9)<br>
    4.2 [FX class](#section10)<br>
    4.3 [Equity class](#section11)<br>
5) [Netting Set class](#section12)<br>


## <a id="section1"></a> 1) Brief overview of the SA CCR methodology and instrument parametors required for computation EAD for each netting set

### <a id="section2"></a>EAD as a function of current exposure (RC) and potential future exposure (PFE)
The purpose of SA CCR is to determine the EAD for each netting set. The EAD is a function of the current exposure of the portfolio (reflected in the replacement cost RC) and the potential future exposure (PFE). 




### <a id="section3"></a> General steps followed for computing AddOn(asset class)
<br>
As mentioned in the introduction, the SA CCR methodology can be split into 3 majors steps. The first step is allocating instruments that constiture the netting set into the corresponding asset class, hedging set and bucket for some categories of asset class. The second step consists in mapping the adjusted notional of the transaction, the supervisory delta and the Maturity factor. The three parametors delta, adjusted notional maturity factor and effective notional are multiplied to obtain the effective notional of the transaction. The third step is a bottom up aggregation of.<br>

<br>
<details>
  <summary>[52.30] : general steps for the calculation of asset class level add on (common to all asset classes) </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.30.png" alt="image"  width="600" />
</details>
<br>

**<br><u>Netting set level</u><br>**
The first node (highest level) contains Netting set level data:
- EADi : the Exposure at Default of the netting set expressed as a function of RC and PFE [52.XX]:
    - RC : expressed as a function of collateral held and aggregated MtM value of transactions V [52.XX]:
        - C : total collateral of the Netting set
        - V : aggregated MtM value of all transactions in the netting set (sum of instrument level MtM)
    - PFE : the potential future exposure of the NS expressed as the product of a multiplier m and the aggregated AddOn:
        - AddOn(aggregated): sum of asset class level add ons AddOn(asset class)
        - m : multiplier expressed as a function of AddOn(Aggregated), colateral C, and total MtM V<br>

**<u>Asset class level</u><br>**
The second (lower level) Node contains Asset class level data:
- AddOn(asset class): calculated as the sum of hedging set level add on AddOn(hedging set)<br>

**<u>Hedging Set level</u><br>**
The third Node contains Hedging set level data: 
- AddOn(hedging set) : expressed as the product of the hedging set level effective notional effN(HS) and a supervisory factor SF:
    - effN(HS) : the hedging set level effective notional expressed as the sum of either bucket level effective notionals (IR and commodity) or instrument level notionals (for equity, fx, credit). The effective notional essentially represents the linear sensitivity (or delta) of transaction to the hedging set risk factors. 
    - SF : the supervisory factor represents the underlying shock applied to the hedging set notional ([52.72])

**<u>Bucket level</u><br>**
Node 4 (for IR class and CTY asset classes only) contains bucket level data:
- effNotional(bucket) : the bucket level effective notional, expressed as a function of transaction level effective notionals <br>

**<u>Instrument/Transaction level</u><br>**
Node 5 contains instrument/transaction level effective notionals:
- Di : the effective notional of the transaction expressed as the product of addNi, δi and MFi
    - di : adjusted notional of the instruments/transaction
    - δi : delta adjustment of the transaction
    - MFi : maturity factor of the transaction:
        - for margined transactinos, MFi is defined at netting set level via MPOR ([52.48])
        - for unmargined transactions, MFi is a function of the maturity of the trade ([52.52])

<br>
<details>
  <summary>[52.72] : supervisory parametors (volatility, SF, correlation) </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.72.png" alt="image"  width="600" />
</details>
<details>
  <summary>[52.48] : maturity factor Mi for unmargined netting sets </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.48.png" alt="image"  width="600" />
</details>

<details>
  <summary>[52.52] : maturity factor Mi for margined netting sets </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.52.png" alt="image"  width="600" />
</details>



## <a id="section4"></a> 2) Intrument superclass and subclasses

The contruction of the instrument superclass, its asset class specific sub-classes and the instrument subclasses of asset class secific instruments is thouroughly detailed in a previous notebook (see "SA_CCR_instrument_mapping.ipynb").
Classes that inherit from the instrument class contain the necessary information to:
- allocate the instrument to the corresponding {asset class x hedging set}
- calculate the MtM value of the instrument (for the purpose of calculating RC) 
- calculate the effective notional (equivalent to the delta sensitivity to the hedging set risk factors). 

<br><br>
**Inheritance structure of the instruments<br>**
<br>
<details>
  <summary>See Jupyter Notebook "SA_CCR_instrument_mapping.ipynb" for detailed construction of these classes</summary>
  <img src="images_SSA_CCR_aggregation_netting_set/instrument_mapping.png" alt="image"  width="600" />
</details>

**Constructing hypothetical portfolio of instruments<br>**
We construct a portfolio of 6 instruments: IR swap, Forward Swap, Swaption, FX Forward, FX Call, Equity Call 


In [50]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 18 12:25:47 2024

@author: joshuakaji
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os 
import copy

os.chdir('/Users/joshuakaji/Desktop/freelance/Interview preparation/CCR')
# Superclasses
from SA_CCR_common import instrument, assetClass, hedgingSet
from SA_CCR_common import position_to_str, get_delta_call
# Asset 
from SA_CCR_EQT import equityInstrument, euroCallEquity, black_scholes_call, equity, equityHedgingSet
from SA_CCR_FX import FXInstrument, fxForward, calculate_forward_value, fx_convert, fxCall, calculate_fx_call, FX, FXHedgingSet
from SA_CCR_IR import IR, IRInstrument, IRSwap, fwdSwap, euroSwaption
from SA_CCR_FX import currency_volatilities, currency_spots, currency_yields
from SA_CCR_common import position_to_str, get_delta_call

##############################################################################
# IR Swap
##############################################################################
IRswap1 = IRSwap(10, 100e3, 1, "USD", K=0.05, freq=0.5)
IRswap2 = IRSwap(5, 100e3, -1, "EUR", K=0.05, freq=0.5)


##############################################################################
# IR Forward Swap
##############################################################################
fwd_swap1 = fwdSwap(5, 10, 100e3, -1, 'JPY', K=0.05, freq=0.5)
fwd_swap2 = fwdSwap(1, 2.5, 100e3, 2, 'USD', K=0.06, freq=0.5)


##############################################################################
# IR Swaption
##############################################################################

r0, K = 0.05, 0.05
euro_swaption1 = euroSwaption(0.5, 5, 100e3, r0-0.01, K, 1, "USD")
euro_swaption2 = euroSwaption(0.5, 10, 100e3, r0, K, 1, "EUR")
euro_swaption3 = euroSwaption(0.5, 10, 100e3, r0+0.01, K, 1, "JPY")

##############################################################################
# FX Forward
##############################################################################

# common params 
T_settlement = 0.5
# Notional in BASE currency
size, N = 1, 1e6

ccy_base, ccy_quote = 'EUR', 'USD'
fx_spot = currency_spots[ccy_base+'/'+ccy_quote]
fwd_rate = fx_spot
fx_fwd1 = fxForward(T_settlement, N, ccy_base, ccy_quote, fx_spot, fwd_rate, size)

ccy_base, ccy_quote = 'EUR', 'GBP'
fx_spot = currency_spots[ccy_base+'/'+ccy_quote]
fwd_rate = fx_spot
fx_fwd2 = fxForward(T_settlement, N, ccy_base, ccy_quote, fx_spot, fwd_rate, size)

ccy_base, ccy_quote = 'GBP', 'JPY'
fx_spot = currency_spots[ccy_base+'/'+ccy_quote]
fwd_rate = fx_spot
fx_fwd3 = fxForward(T_settlement, N, ccy_base, ccy_quote, fx_spot, fwd_rate, size)

##############################################################################
# FX Call
##############################################################################
# common params
N = 1e6
T_exercise=3/12
size = 1
fx_pair='EUR/USD'
fx_spot = currency_spots[fx_pair]
# set strick as 5% OTM
stike = fx_spot*0.95
sigma = currency_volatilities[fx_pair]
# call on EUR/USD with exercise date in 3 months, 5% out of the money
fx_call1 = fxCall(T_exercise, N, fx_pair, fx_spot, stike, size)
fx_pair='GBP/USD'
fx_spot = currency_spots[fx_pair]
# set strick as 5% OTM
stike = fx_spot*1.05
sigma = currency_volatilities[fx_pair]
fx_call2 = fxCall(T_exercise, N, fx_pair, fx_spot, stike, size)


##############################################################################
# Equity Call (European)
##############################################################################
S0 = 90
K = 100
T = 0.5
N = 1000 # 1000 undelyings in per option/instrument
name = 'aapl'
single_name = True
sigma = 0.3
euroCall1 = euroCallEquity(S0, K, T, name, 1, single_name, N)
name = 'tesla'
euroCall2 = euroCallEquity(S0, K, T, name, -2, single_name, N)

name = 'tesla'
euroCall3 = euroCallEquity(S0, K, T, name, -1, single_name, N)

name = 'snp500'
single_name = False
euroCall4 = euroCallEquity(S0, K, T, name, -2, single_name, N)

# assume unmargined netting set 
MPOR = np.nan
L = [IRswap1, fwd_swap1, euro_swaption1,fx_fwd1, fx_call1, euroCall1]
L2 = [IRswap1, IRswap2, fwd_swap1, euro_swaption1, euro_swaption2, euro_swaption3, fx_fwd1, fx_fwd2, fx_call1, fx_call2, euroCall1, euroCall2]
for instr in L:
    instr.set_delta()
    instr.set_MF(MPOR)
    instr.set_Di()
    print(instr, '\nMtM : %.2f €'% instr.value)
    print('di : %.2f'% instr.di)
    print('δi: %.2f'% instr.delta)
    if instr.asset_class=='IR':
        print('Maturity Bucket : %i'% instr.bucket)
    print('Hedging Set : %s\n'% instr.hs)

IR swap maturing in 10 years // long 1 
MtM : -11307.93 €
di : 786938.68
δi: 1.00
Maturity Bucket : 3
Hedging Set : USD

10-year interest rate swap, forward starting in 5 years // short 1 
MtM : -6570.25 €
di : 612868.46
δi: -1.00
Maturity Bucket : 3
Hedging Set : JPY

Cash-settled European swaption referencing 5-year interest rate swap with exercise date in 0.5 years // long 1 
MtM : 0.00 €
di : 431475.58
δi: 0.32
Maturity Bucket : 1
Hedging Set : USD

FX FWD to buy 1000000 EUR (base) against USD (quote) at forward rate 1.2 in 0.5 years // long 1 
MtM : 6304.64 €
di : 1000000.00
δi: 1.00
Hedging Set : EUR/USD

FX call on EUR/USD with strike 1.14 with exercise in 0.2 years on (N = 1000000 EUR // long 1 
MtM : 53238.30 €
di : 1000000.00
δi: 0.76
Hedging Set : EUR/USD

euro call on aapl with exercise date in 0.5 years // long 1 
MtM : 79974.12 €
di : 90000.00
δi: 0.62
Hedging Set : aapl



## <a id="section5"></a> 3) Hedging Set superclass and asset class specific subclasses

At the hedging set level, the effective notional of transactions are aggregated and the hedging set level add-ons calculated as the product of the supervisory factor of that hedgnig set (provided in table [52.72]). hedging set level Effective Notionals (Dj) are essentially the sensitivity of those instruments to the hedging set risk factors . The supervisory factors are a conservative shock (calibrated over one year) that one can expect during PFE horizon. The Add-On calculated is therefore a conservative estimate of the portfolios PnL following market changes. The variable of interest that must be calculated to proceed to the next level (asset class level aggregation) is the add-on by hedging set AddOn(hedging set).





In [25]:
class hedgingSet():
    def __init__(self, L): # list of instruments that ahve been defined
        # check that all intrusments have the same HS
        if len(np.unique([instr.hs for instr in L])) > 1:
            raise ValueError("Instantiating HS object requires instruments to be from same HS")
        self.L = L
        # calculate MtM of all instruments in Hedging Set 
        self.mtm = sum([instr.value for instr in self.L])


### <a id="section6"></a> 3.1) IR Hedging Set class


Hedging sets are defined according to currencies (eg : EUR, USD, JPY). 
Buckets are futher definedaccording to maturity of the instrument (below 1Y, between 1Y and 5Y, above 5Y). The maturity is defined as the maximum lifetime of the contract as defined in [52.31 (1)].
For interest rate instruments, full offset is permitted within maturity buckets (below 1Y, between 1Y and 5Y, above 5Y). The aggregation across buckets within a hedging set is partial and assumes a correlation structure between instruments from different asset classes. 


<br>
<details>
  <summary>[52.57] : aggregation in IR hedging set </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.57.png" alt="image"  width="600" />
</details>



In [27]:
# [52.72] supervisory factor table
class IRHedgingSet():
    def __init__(self, L): # list of instruments that ahve been defined
        hedgingSet.__init__(self, L)
        # allocate instruments into buckets
        self.buckets = {}
        for b in [1,2,3]:
            L_bucket = [inst for inst in L if inst.bucket == b]
            self.buckets[b] = IRBucket(L_bucket)
        # calculate effective notional of Hedging set
        Djx = np.matrix[self.buckets]
        # [52.57 (5)] : effective notional of the hedging set
        self.effNj = np.sqrt(self.buckets[1].Djk**2 +self.buckets[2].Djk**2 + self.buckets[3].Djk**2 \
            + 1.4*self.buckets[1].Djk*self.buckets[2].Djk + 1.4*self.buckets[2].Djk*self.buckets[3].Djk \
                +0.6*self.buckets[1].Djk*self.buckets[3].Djk)
        # calculate AddOnj of the HS (supervisory factor for IR is 0.5%)
        # [52.57 (6)] : Calculate the hedging set level add-on (SF for IR is 0.5%)
        SF = 0.005
        self.AddOnj = SF * self.effNj

class IRBucket():
    def __init__(self, L):
        # check that all intrusments have the same bucket
        if len(np.unique([inst.bucket for inst in L])) > 1:
            raise ValueError("Instantiating bucket object requires instruments to be from same bucket")
        self.L = L
        # calculate effective notional of bucket k, HS j
        # [52.57 (1)] : the effective notional Di is calculated as Di = di * MFi * δi
        # [52.57 (4)] : effective notional of a maturity bucket
        self.Djk = sum([instr.delta * instr.di * instr.MF for instr in L])
        # calculate MtM of all instruments in bucket 
        self.mtm = sum([instr.value for instr in self.L])

### <a id="section6"></a> 3.2) FX Hedging Set class


FX Hedging sets are defined according to currencie pairs (eg : EUR/USD, USD/GBP, JPY/USD).
Full offset is permitted between same currency pairs to calculate the hedging set level effective notional Dj. 
The hedging set level add-on is calculated as the product of the absolute effective notional and the supervisory factors SF (0.04% for across all currency pairs).


<br>
<details>
  <summary>[52.59] : FX aggregation </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.59.png" alt="image"  width="600" />
</details>



In [28]:

class FXHedgingSet(hedgingSet):
    def __init__(self, L): # list of instruments that ahve been defined
        hedgingSet.__init__(self, L)
        # calculate effective notional of Hedging set
        self.effNj = sum([instr.di * instr.delta * instr.MF for instr in L])
        # [52.59 (4)] : Calculate the hedging set level add-on (SF for FX is 4%)
        SF = 0.04
        self.AddOnj = abs(self.effNj) * SF

### <a id="section7"></a> 3.3) Equity Hedging Set class


Equity Hedging sets are defined according to single name entities (eg: TESLA, BNP, AAPL) or indices (eg: SNP500, NASDAQ, DJIA). 

Full offset is permitted between instruments with same-name entities to calculate hedging set level add-on. The supervisory factors applied are 32% for single names and 20% for indices.


<br>
<details>
  <summary>[52.66] : Equity aggregation </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.66.png" alt="image"  width="600" />
</details>



In [29]:
class equityHedgingSet(hedgingSet):
    def __init__(self, L):
        self.single_name = L[0].single_name 
        hedgingSet.__init__(self, L)
        # [52.66 (1) & (2)] calculate effective notional of Hedging set
        self.effNj = sum([instr.di * instr.delta * instr.MF for instr in L])
        # [52.66 (3)] : Calculate the hedging set entity level add-on 
        # [52.72] : SF for Equity depends on whether underlying is index or single name 
        SF = 0.32 if self.single_name else 0.2
        self.AddOnj = self.effNj * SF


## <a id="section8"></a> 4) Asset class superclass and asset class specific sub-classes

At the asset class level, the aggregation of the hedging set level add-ons is conducted in order to obtain the asset class level add-on. 
SA CCR distinguishes two methods for aggregating results: 
- simply summing the hedging set level add-ons: this is used for IR, FX and Commodity
- aggregation of hedging set level add-ons assuming a 1 factor correlation model (for Credit and Equity). ie: all hedging level variations are assumed to come in part (proportional to supervisory correlation factor prescribed) from the movements of single systemic factor.

<br>
<details>
  <summary>[52.54] : supervisory correlation parameter </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.54.png" alt="image"  width="600" />
</details>


In [30]:

class assetClass():
    # when defining a HS, we must give margin agreement rules (ie : margin or no)
    def __init__(self, L, MPOR):
        self.MPOR = MPOR
        self.L = L
        # set delta and MF for all insturments
        for instr in self.L:
            instr.set_MF(self.MPOR)
            instr.set_delta()
            instr.set_Di()
        # calculate MtM of all instruments in Asset Class
        self.mtm = sum([instr.value for instr in self.L])
    # view AddOnj (ie: HS AddOn) for all HSs
    def view_HS_contribution(self, plot=True):
        summary = {}
        for k in self.HS.keys():
            summary[k] = self.HS[k].AddOnj
        df = pd.DataFrame(list(summary.items()), columns=['HS', 'AddOn(HS)'])
        if plot:
            df.plot(kind='bar', x='HS', y='AddOn(HS)', grid=True)
        return df
    def view_instruments_contribution(self, plot=True):
        data = []
        count = 1
        for instr in self.L: 
            data.append([count, str(instr), instr.di, instr.delta, instr.MF, instr.Di, instr.value])
            count+= 1
        df = pd.DataFrame(data, columns=['instrument', 'description', 'di', 'delta', 'MF', 'Di', 'mtm'])
        return df



### <a id="section10"></a> 4.1) IR asset class subclass

The asset class level add-on for IR is simply the sum of hedging set level add-ons.

In [31]:

class IR(assetClass):
    def __init__(self, L, MPOR):
        assetClass.__init__(self, L, MPOR)
        # allocate instruments into hedging sets
        self.HS = {}
        # [52.57 (2)] for IR, HS correspond to currencies
        for hs in np.unique([instr.hs for instr in self.L]):
            self.HS[hs] = IRHedgingSet([instr for instr in self.L if instr.hs==hs])
        # calculate the AddOn(asset class)
        # [52.57(7)] : IR
        self.AddOn = sum([hs.AddOnj for hs in self.HS.values()])
    
    def view_bucket_contribution(self, hs='ALL', plot=True, with_aggregate=True):
        if (type(hs) == str) & (hs !='All'):
            summary = {}
            for b in self.HS[hs].buckets.keys():
                summary[b] = self.HS[hs].buckets[b].Djk
            df = pd.DataFrame(list(summary.items()), columns=['bucket', 'effN'])
            if plot:
                df.plot(kind='bar', x='bucket', y='effN', legend=False, grid=True)
            return df
        if (type(hs) == list) | (hs=='All'):
            data = []
            if hs=='All':
                hs = list(self.HS.keys())
            for k in hs:
                for b in self.HS[k].buckets.keys():
                    data.append([k, b, self.HS[k].buckets[b].Djk])
                if with_aggregate:
                    data.append([k, 'aggregat', self.HS[k].effNj])
            df = pd.DataFrame(data, columns=['HS', 'bucket', 'effN'])
            if plot:
                df.pivot(index='HS', columns='bucket', values='effN').plot(kind='bar', grid= True)
            return df



### <a id="section10"></a> 4.2) FX asset class subclass

The asset class level add-on for IR is simply the sum of hedging set level add-ons.

In [32]:

class FX(assetClass):
    def __init__(self, L, MPOR):
        # allocate instruments into hedging sets
        self.HS = {}
        assetClass.__init__(self, L, MPOR)
        # [52.59 (2)] for FX, hedging sets correspond to currencies
        for hs in np.unique([instr.hs for instr in self.L]):
            self.HS[hs] = FXHedgingSet([instr for instr in self.L if instr.hs==hs])
        # calculate the AddOn(asset class)
        # [52.57(7)] : IR
        self.AddOn = sum([hs.AddOnj for hs in self.HS.values()])



### <a id="section11"></a> 4.3) Equity asset class subclass

The asset class level aggregation is made assuming a 1 factor correlation model. The supervisory correlations factors applied are respectively 80% and 50% for indices and single names entities. The higher correlation of indices to the systemic factor reflects the fact that indices diversify away indioyncraties of the single names that compose it.

In [33]:

class equity(assetClass):
    def __init__(self, L, MPOR):
        # allocate instruments into hedging sets
        self.HS = {}
        assetClass.__init__(self, L, MPOR)
        # [52.59 (2)] for equity, hedging sets correspond to entities
        for hs in np.unique([instr.hs for instr in self.L]):
            self.HS[hs] = equityHedgingSet([instr for instr in self.L if instr.hs==hs])
        # calculate the AddOn(asset class)
        # [52.56 (4)] : calculate AddOnEquity by aggregate across entities/HS 
        # [52.72] : systematic correlation param depends on whether index or single name
        rho_single_name = 0.5
        rho_index = 0.8
        sum1_single_names = [hs.AddOnj * rho_single_name for hs in self.HS.values() if hs.single_name]
        sum1_index = [hs.AddOnj * rho_index for hs in self.HS.values() if hs.single_name]
        sum1 = sum(sum1_single_names + sum1_index)**2
        sum2_single_names = sum([hs.AddOnj * (1 - rho_single_name**2) for hs in self.HS.values() if hs.single_name])
        sum2_index = sum([hs.AddOnj**2 * (1 - rho_index**2) for hs in self.HS.values() if not hs.single_name])
        sum2 = sum2_index + sum2_single_names
        self.AddOn = np.sqrt(sum1 + sum2)


## <a id="section12"></a> 3) Netting Set class

**The Netting Set class is instantiated using the following information/parameters:**
- A list of instruments (as defined in the previous section)
- Margin Agreement (MA) specificities if one is in place: 
    - MPOR (Margin period of risk): Estimated by the bank, representing the time between last margin call and and closeout or hedging of the portfolio after default of the counterparty.
    - threshold : margin calls are calculated as the amount above this threshold. The threshold represents the exposure that the bank is to tolerate before requiring collateral
    - Minimum transfer amount (MTM) : margin calls are not exchanged when the amount is below the Minimum Transfer Amount
- Collateral (posted and recieved):
    - collateral obtained from margin calls to off-set exposure of the portfolio
    - Net Independant Colateral amount (NICA) : collateral that is independant of the Mark to market value of the portfolio. Calculated as the collateral held by the bank minus the collateral provided by the bank to the counterparty (excepting collateral held in a segregated account, ie: escrow):
        - Independant Amount (IA) : contractually agreed upon at onset of netting agreement
        - Initial Margin (IM): estimated by both counterparties, usually via industry standard methods (ISDA Model SIMM), its purpose is to cover the change in value of the portfolio during time MPOR due to market changes. It is essentially designed to cover the Potential future Exposure (PFE). The estimation of IM is made not from the MtM value of the portfolio per se but rather the paramteters/sensitivities of the portfolio  to various risk factors. 

**Calculation of the EAD of the Netting Set**
<br><br>
<details>
  <summary>[52.1] : EAD requires computation of Replacement cost (RC) and potential future exposure (PFE) </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.1.png" alt="image"  width="600" />
</details>

<u>Note</u>: the alpha parameter is applied to account for model inacuracies. Litterature outside of BCBS documentation indicate that it accounts (in part) for wrong way risk (wrong way risk). Mathematically, WWR is the positive correlation between the default of a counterparty (PD) and the exposure (EAD) that the bank has with that counterparty (ie : when the counterparty expriences downgrade, the exposure tends to be higher). Typical examples of WWR include long credit protection (CDS) with a counterparty whose default is correlated to the underlying entity.
<br><br>

<details>
  <summary>[52.10] : Replacement Cost (RC) for unmargined transactions </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.10.png" alt="image"  width="600" />
</details>

<details>
  <summary>[52.18] : Replacement Cost (RC) for margined transactions </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.18.png" alt="image"  width="600" />
</details>


<details>
  <summary>[52.20] : PFE potential Future Exposure </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.20.png" alt="image"  width="600" />
</details>

The multiplier takes into account excess colateral by scaling down the PFE when the portfolio value (V-C) is negative via the formula below. 
<br><br>
<details>
  <summary>[52.23] : multiplier depends on V, C, and aggregated Add On </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.23.png" alt="image"  width="600" />
</details>

The aggregate add on is the sum of add ons calculated for each of the 5 asset classes: IR, FX, Equity, Credit, Commodity
<br>
<details>
  <summary>[52.25] : aggregate add on formula </summary>
  <img src="images_SSA_CCR_aggregation_netting_set/52.25.png" alt="image"  width="600" />
</details>




In [101]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 14 13:48:22 2024

@author: joshuakaji
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os 
import copy
import os 
# os.chdir('/Users/joshuakaji/Desktop/freelance/Interview preparation/CCR')
from SA_CCR_IR import IR
from SA_CCR_FX import FX
from SA_CCR_EQT import equity
# [52.31] Time period parameters: Mi, Ei, Si and Ti




class nettingSet():
    def __init__(self, L, MPOR, C, mta=0, th=0):
        self.C = C # look up collateral HC prescribed by BCBS 
        # NICA represents the IM and independant amount 
        # unsegregated collateral posted by the bank is not taken into account since we assume that it will be given back in ncase of ctp default.
        self.NICA = C # look up collateral HC prescribed by BCBS 
        self.L = copy.deepcopy(L)
        self.MPOR = MPOR  
        self.L_ir = []
        self.L_fx = []
        self.L_equity = []
        
        for instr in self.L:
            if instr.asset_class == 'IR':
                self.L_ir.append(instr)
            if instr.asset_class == 'FX':
                self.L_fx.append(instr)
            if instr.asset_class == 'Equity':
                self.L_equity.append(instr)
        # create asset level aggregation nodes
        self.aggregated_nodes = {}
        self.aggregated_nodes['IR'] = IR(self.L_ir, self.MPOR)
        self.aggregated_nodes['FX'] = FX(self.L_fx, self.MPOR)
        self.aggregated_nodes['Equity'] = equity(self.L_equity, self.MPOR)
        
        # calculate AddOn at netting set level
        self.AddOnAggregated = sum([node.AddOn for node in self.aggregated_nodes.values()])
        # calculate V (MtM of the netting)
        self.V = sum([instr.value for instr in self.L if not np.isnan(instr.value) ])
        # [52.23] : calculate multiplier
        floor = 0.05
        self.m = min(1, floor + (1 - floor)*np.exp((self.V - self.C)/(2 * (1-floor)* self.AddOnAggregated)))
        # [52.20] calculate PFE
        self.PFE = self.m * self.AddOnAggregated
        # [52.3 to 52.19] : replacement cost RC
        if np.isnan(MPOR): # [52.10] for Unmargined NS
            # [52.17] : C = NICA (Net Independant Collateral Amount)
            self.RC = max(self.V - self.C, 0)
        else:
            # [52.18] : for Margined NS
            self.RC = max(self.V - self.C, th + mta - self.NICA, 0)
        # [52.1] : EAD of the netting set, alpha = 1.4 for model innacuracies (eg: wrong way risk)
        self.EAD = 1.4 * (self.RC + self.PFE)
        
    def view_asset_class_contribution(self, plot=True):
        summary = {}
        for k in self.aggregated_nodes.keys():
            summary[k] = self.aggregated_nodes[k].AddOn
        df = pd.DataFrame(list(summary.items()), columns=['asset class', 'AddOn(asset class)'])
        if plot:
            df.plot(kind='bar', x='asset class', y='AddOn(asset class)', grid=True)
        return df
    def view_instruments_contribution(self, plot=True):
        data = []
        count = 1
        for instr in self.L: 
            data.append([count, str(instr), instr.di, instr.delta, instr.MF, instr.Di, instr.value])
            count+= 1
        df = pd.DataFrame(data, columns=['instrument', 'description', 'di', 'delta', 'MF', 'Di', 'mtm'])
        return df

    def view_hedging_set_contribution(self, asset_class='ALL', plot=True, with_aggregate=False):
        if (type(asset_class) == str) & (asset_class !='All'):
            summary = {} 
            for hs in self.aggregated_nodes[asset_class].HS.keys():
                summary[hs] = self.aggregated_nodes[asset_class].HS[hs].AddOnj
            df = pd.DataFrame(list(summary.items()), columns=['Hedging Set', 'AddOn(HS)'])
            if plot:
                df.plot(kind='bar', x='bucket', y='effN', legend=False, grid=True)
            return df
        if (type(asset_class) == list) | (asset_class=='All'):
            data = []
            if asset_class=='All':
                asset_classes = list(self.aggregated_nodes.keys())
            for a in asset_classes:
                for hs in self.aggregated_nodes[a].HS.keys():
                    data.append([a, hs, self.aggregated_nodes[a].HS[hs].AddOnj, self.aggregated_nodes[a].HS[hs].mtm])
                if with_aggregate:
                    data.append([a, 'aggregat', self.aggregated_nodes[a].AddOn])
            df = pd.DataFrame(data, columns=['asset class', 'hedging set', 'AddOn', 'MtM'])
            if plot:
                df.pivot(index='asset class', columns='hedging set', values='AddOn').plot(kind='bar', grid= True)
                #df.pivot(index='asset class', columns='hedging set', values='MtM').plot(kind='bar', grid= True, ax =ax[1] )
            return df



**Test on hypothetical portfolio**<br>


In [102]:

##############################################################################
# Create Netting set of instruments 
##############################################################################

NS1 = nettingSet(L2, MPOR=10, C=0, mta=0, th=0)
NS2 = nettingSet(L2, MPOR=20, C=0, mta=0, th=0)
NS3 = nettingSet(L2, MPOR=np.nan, C=0)
df = NS1.view_hedging_set_contribution('All', plot=False)
df2 = NS2.view_hedging_set_contribution("All", plot=False)
df3 = NS3.view_hedging_set_contribution("All", plot=False)
print("#"*25, ' MPOR = 10 ', "#"*25)
print("PFE : ", NS1.PFE)
print("RC : ", NS1.RC)
print("EAD : ", NS1.EAD)
print(df1)
print("#"*25, ' MPOR = 20 ', "#"*25)
print("PFE : ", NS2.PFE)
print("RC : ", NS2.RC)
print("EAD : ", NS2.EAD)
print(df2)
print("#"*25, ' Unmargined NS ', "#"*25)
print("PFE : ", NS3.PFE)
print("RC : ", NS3.RC)
print("EAD : ", NS3.EAD)
print(df3)

#########################  MPOR = 10  #########################
PFE :  46855.79644995862
RC :  204413.30369510886
EAD :  351776.7402030945
  asset class hedging set         AddOn           MtM
0          IR         EUR    780.944045  -6658.180970
1          IR         JPY   1059.301673  -6570.254191
2          IR         USD   1259.533130 -11307.931987
3          FX     EUR/GBP  12000.000000   8062.664521
4          FX     EUR/USD  21176.058409  59542.935518
5          FX     GBP/USD   3639.457430   1395.839549
6      Equity        aapl   5339.069408  79974.115628
7      Equity       tesla -10678.138815  79974.115628
#########################  MPOR = 20  #########################
PFE :  66264.22230424487
RC :  204413.30369510886
EAD :  378948.5363990952
  asset class hedging set         AddOn           MtM
0          IR         EUR   1104.421660  -6658.180970
1          IR         JPY   1498.078793  -6570.254191
2          IR         USD   1781.248835 -11307.931987
3          FX     EU